# Download YouTube Video

In [ ]:
!pip install pytube
!pip install pydub
from pytube import YouTube
from pydub import AudioSegment
import librosa
import soundfile as sf
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def download_yt_wav(link, video_path, file_name):
  """
  Downloads an mp4 file of a YouTube video, transforms it into a wav file and sets the sample rate.
  """

  # create a YouTube object of the video from the link in order to use this library
  try: 
    yt = YouTube(link) 
  except: 
      print("Connection Error")
  
  # use this to check out different stream options for mp4 download such as resolution and fps, pick the tag with the options we want for the download
  #yt.streams.filter(file_extension='mp4')

  # we want option 139: <Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">
  youtube = YouTube(link)
  audio_stream = youtube \
    .streams \
    .filter(progressive=True, file_extension='mp4') \
    .order_by('bitrate') \
    .last()
  audio_stream.download(
    output_path=os.path.dirname("video_path"),
    filename=os.path.basename(video_path + '/'+ file_name + '.mp4'))

  # take the saved mp4 file
  given_audio = AudioSegment.from_file(video_path + '/'+ file_name + '.mp4', format="mp4")   

  # and transform it into a wav file with the same name
  given_audio.export(file_name + ".wav", format="wav")

  # read the wav file, change the sample rate to 16000 and write that file to replace the old one
  sp, rate = sf.read(file_name + ".wav")
  sp= librosa.resample(sp.T, rate, 16000)
  sf.write(file_name + ".wav", sp.T, 16000, subtype='PCM_24')

In [ ]:
!pip install youtube-dl pydub

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import os
import re
import torch
import torchaudio

import IPython

from dataclasses import dataclass
from pydub import AudioSegment

In [ ]:
# video_id = "dQw4w9WgXcQ" # rick roll
# video_id = "7wtfhZwyrcc" # believer
video_id = "b1kbLwvqugk" # taylor
# video_id = "dNCWe_6HAM8" # money 
# video_id = "ru0K8uYEZWw" # CAN'T STOP THE FEELING!
# video_id = "XbGs_qK2PQA" # rap god

In [ ]:
# Download the video using youtube-dl
os.system("youtube-dl --extract-audio --audio-format wav --audio-quality 0 -o '%(id)s.%(ext)s' https://youtu.be/{}".format(video_id))

In [ ]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H

In [ ]:
def get_wave(aud):
  aud = aud.set_channels(1)
  aud = aud.get_array_of_samples()
  wave = torch.tensor(aud, dtype = torch.float)
  wave = torch.reshape(wave, (1,wave.shape[0]))

  return wave

In [ ]:
# Load the audio file using pydub
audio = AudioSegment.from_file("{}.wav".format(video_id), format="wav")

waveform = get_wave(audio)
print(waveform.shape)

sr = audio.frame_rate
print(sr)

In [ ]:
file_path = "{}.wav".format(video_id)
if os.path.isfile(file_path):
    os.remove(file_path)
    print("{} has been deleted.".format(file_path))
else:
    print("{} does not exist.".format(file_path))


In [ ]:
if sr != bundle.sample_rate:
    waveform = torchaudio.functional.resample(waveform, sr, bundle.sample_rate)

In [ ]:
print(torch.__version__)
print(torchaudio.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# %matplotlib inline

matplotlib.rcParams["figure.figsize"] = [16.0, 4.8]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model = bundle.get_model().to(device)
labels = bundle.get_labels()
length = waveform.shape[1]
chunks = []
amount_chunks = 10
chunks_length = length//amount_chunks
for i in range(amount_chunks):
  with torch.inference_mode():
      emissions, _ = model(waveform[:, i * chunks_length: min(length, (i + 1) * chunks_length)].to(device))
      emissions = torch.log_softmax(emissions, dim=-1)
      chunks.append(emissions)

emission = torch.cat(chunks, dim=1)[0].cpu().detach()

In [ ]:
rick_roll_lyrics = """

[Verse 1]
We're no strangers to love
You know the rules and so do I
A full commitment's what I'm thinking of
You wouldn't get this from any other guy

[Pre-Chorus]
I just wanna tell you how I'm feeling
Gotta make you understand

[Chorus]
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you

[Verse 2]
We've known each other for so long
Your heart's been aching, but you're too shy to say it
Inside, we both know what's been going on
We know the game, and we're gonna play it

[Pre-Chorus]
And if you ask me how I'm feeling
Don't tell me you're too blind to see

[Chorus]
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you

[Post-Chorus]
Ooh (Give you up)
Ooh-ooh (Give you up)
Ooh-ooh
Never gonna give, never gonna give (Give you up)
Ooh-ooh
Never gonna give, never gonna give (Give you up)

[Bridge]
We've known each other for so long
Your heart's been aching, but you're too shy to say it
Inside, we both know what's been going on
We know the game, and we're gonna play it

[Pre-Chorus]
I just wanna tell you how I'm feeling
Gotta make you understand

[Chorus]
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you"""

In [ ]:
believer_lyrics = """[Verse 1]
First things first
I'ma say all the words inside my head
I'm fired up and tired of
The way that things have been, oh-ooh
The way that things have been, oh-ooh

[Verse 2]
Second things second
Don't you tell me what you think that I could be
I'm the one at the sail
I'm the master of my sea, oh-ooh
The master of my sea, oh-ooh

[Pre-Chorus]
I was broken from a young age
Taking my sulkin' to the masses
Writing my poems for the few
That look at me, took to me, shook at me, feelin' me
Singing from heartache from the pain
Taking my message from the veins
Speaking my lesson from the brain
Seeing the beauty through the...

[Chorus]
Pain! You made me a, you made me a
Believer, believer
Pain! You break me down, you build me up
Believer, believer
Pain! Oh, let the bullets fly, oh, let them rain
My life, my love, my drive, it came from…
Pain! You made me a, you made me a
Believer, believer

[Verse 3]
Third things third
Send a prayer to the ones up above
All the hate that you've heard
Has turned your spirit to a dove, oh-ooh
Your spirit up above, oh-ooh

[Pre-Chorus]
I was chokin' in the crowd
Building my rain up in the cloud
Falling like ashes to the ground
Hoping my feelings, they would drown
But they never did, ever lived, ebbin' and flowin'
Inhibited, limited 'til it broke open and rained down
It rained down, like…

[Chorus]
Pain! You made me a, you made me a
Believer, believer
Pain! You break me down, you build me up
Believer, believer
Pain! Oh, let the bullets fly, oh, let them rain
My life, my love, my drive, it came from…
Pain! You made me a, you made me a
Believer, believer

[Verse 4]
Last things last
By the grace of the fire and the flames
You're the face of the future
The blood in my veins, oh-ooh
The blood in my veins, oh-ooh

[Pre-Chorus]
But they never did, ever lived, ebbin' and flowin'
Inhibited, limited 'til it broke open and rained down
It rained down, like…

[Chorus]
Pain! You made me a, you made me a
Believer, believer
Pain! You break me down, you build me up
Believer, believer
Pain! Oh, let the bullets fly, oh, let them rain
My life, my love, my drive, it came from…
Pain! You made me a, you made me a
Believer, believer"""

In [ ]:
taylor_lyrics = """I have this thing where I get older but just never wiser
Midnights become my afternoons
When my depression works the graveyard shift
All of the people I've ghosted stand there in the room
I should not be left to my own devices
They come with prices and vices
I end up in crisis (tale as old as time)
I wake up screaming from dreaming
One day I'll watch as you're leaving
'Cause you got tired of my scheming
(For the last time)
It's me, hi, I'm the problem, it's me
At tea time, everybody agrees
I'll stare directly at the sun but never in the mirror
It must be exhausting always rooting for the anti-hero
Sometimes I feel like everybody is a sexy baby
And I'm a monster on the hill
Too big to hang out, slowly lurching toward your favorite city
Pierced through the heart, but never killed
Did you hear my covert narcissism I disguise as altruism
Like some kind of congressman? (Tale as old as time)
I wake up screaming from dreaming
One day I'll watch as you're leaving
And life will lose all its meaning
(For the last time)
It's me, hi, I'm the problem, it's me (I'm the problem, it's me)
At tea time, everybody agrees
I'll stare directly at the sun but never in the mirror
It must be exhausting always rooting for the anti-hero
I have this dream my daughter in-law kills me for the money
She thinks I left them in the will
The family gathers 'round and reads it and then someone screams out
"She's laughing up at us from hell"
It's me, hi, I'm the problem, it's me
It's me, hi, I'm the problem, it's me
It's me, hi, everybody agrees, everybody agrees
It's me, hi (hi), I'm the problem, it's me (I'm the problem, it's me)
At tea (tea) time (time), everybody agrees (everybody agrees)
I'll stare directly at the sun but never in the mirror
It must be exhausting always rooting for the anti-hero"""

In [ ]:
money_lyrics = """[Verse 1]
It's the end of the month and the weekend
I'ma spend this check, everything on me, yeah
I'ma tip myself, I'ma spend it on myself
I'ma drop it like it's pourin', I'ma pour it on myself

[Refrain]
Che-che-che-check that money-makin' bank account number (Yikes)
That's that shit that's never gettin' bounced on ya
Bitch, I do the money dance, I just made a hundred bands
When the store says, "Sign for it," I'ma leave my autograph

[Pre-Chorus]
Dolla' bills, dolla' bills
Watch it fallin' for me, I love the way that feels
Dolla' bills, dolla' bills
Keep on fallin' for me, I love the way it feels

[Chorus]
I came here to drop some money, droppin' all my money
Drop some money, all this bread so yummy, yeah
Twerkin', twerkin' when I buy the things I like
Dolla', dollas droppin' on my ass tonight

[Verse 2]
Everyone silent, listen to my money talk
Spend how I like it, yeah, everyone know what I mean, mean
When they see green, when they see green, that mean go
Give me what the hell I want, give me what the hell I want

[Refrain]
Check that money-makin' bank account number (Yikes)
That's that shit that's never gettin' bounced on ya
Bitch, I do the money dance, I just made a hundred bands
When the store says, "Sign for it," I'ma leave my autograph

[Pre-Chorus]
Dolla' bills, dolla' bills
Watch it fallin' for me, I love the way that feels
Dolla' bills, dolla' bills
Keep on fallin' for me, I love the way it feels

[Chorus]
I came here to drop some money, droppin' all my money
Drop some money, all this bread so yummy, yeah
Twerkin', twerkin' when I buy the things I like
Dolla', dollas droppin' on my ass tonight
(Drop some money, droppin' all my money)
(Drop some money, all this bread so yummy, yeah)
Drop some money, droppin' all my money
Drop some money, all this bread so yummy, yeah

[Outro]
My money moves, money I choose
CELINE my shoes, walkin' on you, my money rules
My money moves, money I choose
Watch how my wons and yens and dollars be droppin' on you, you, you
Dun, dun-dun-dun-dun-dun, dun
Dun-dun-dun-dun-dun-dun
Dun, dun-dun-dun-dun-dun, dun
Droppin' on you
Dun, dun-dun-dun-dun-dun, dun
Dun-dun-dun-dun-dun-dun
Dun, dun-dun-dun-dun-dun, dun
Droppin' on you"""

In [ ]:
feeling_lyrics = """[Verse 1]
I got this feelin' inside my bones
It goes electric, wavey when I turn it on
All through my city, all through my home
We're flyin' up, no ceilin', when we in our zone

[Refrain]
I got that sunshine in my pocket, got that good soul in my feet
I feel that hot blood in my body when it drops (Ooh)
I can't take my eyes up off it, movin' so phenomenally
Room on lock, the way we rock it, so don't stop

[Pre-Chorus]
And under the lights when everything goes
Nowhere to hide when I'm gettin' you close
When we move, well, you already know
So just imagine (Just imagine, just imagine)

[Chorus]
Nothin' I can see but you
When you dance, dance, dance
A feelin' good, good, creepin' up on you
So just dance, dance, dance, come on
All those things I shouldn't do
But you dance, dance, dance
And ain't nobody leavin' soon, so keep dancin'

[Post-Chorus]
I can't stop the feelin'
So just dance, dance, dance
I can't stop the feelin'
So just dance, dance, dance, come on

[Verse 2]
Ooh, it's somethin' magical
It's in the air, it's in my blood, it's rushin' on (Rushin' on)
I don't need no reason, don't need control (Need control)
I fly so high, no ceilin', when I'm in my zone

[Refrain]
'Cause I got that sunshine in my pocket, got that good soul in my feet
I feel that hot blood in my body when it drops (Ooh)
I can't take my eyes up off it, movin' so phenomenally
Room on lock, the way we rock it, so don't stop

[Pre-Chorus]
And under the lights when everything goes
Nowhere to hide when I'm gettin' you close
When we move, well, you already know
So just imagine (Just imagine, just imagine)

[Chorus]
Nothin' I can see but you
When you dance, dance, dance
A feelin' good, good, creepin' up on you
So just dance, dance, dance, come on
All those things I shouldn't do
But you dance, dance, dance (No, no, no)
And ain't nobody leavin' soon, so keep dancin'

[Post-Chorus]
I can't stop the feelin'
So just dance, dance, dance
I can't stop the feelin'
So just dance, dance, dance, come on
I can't stop the feelin' (Woo)
So just dance, dance, dance
I can't stop the feelin' (Yeah)
So keep dancin', come on

[Bridge]
Oh
Yeah, oh
I can't stop the
I can't stop the
I can't stop the, I can't stop the

[Chorus]
(I can't stop the feelin')
Nothin' I can see but you
When you dance, dance, dance
(I can't stop the feelin')
A feelin' good, good, creepin' up on you
So just dance, dance, dance, come on (Oh)
(I can't stop the feelin')
All those things I shouldn't do
But you dance, dance, dance (Dance, dance, dance)
(I can't stop the feelin')
And ain't nobody leavin' soon, so keep dancin'
(I can't stop the feelin')

[Outro]
Everybody sing
Got this feelin' in my body
(I can't stop the feelin')
Got this feelin' in my body (Oh, no)
(I can't stop the feelin')
Wanna see you move your body (No, no, no)
(I can't stop the feelin')
Got this feelin' in my body, come on
Break it down
Got this feelin' in my body (Uh)
I can't stop the feelin'
Got this feeling in my body, come on (Ooh)"""

In [ ]:
rap_god_lyrics = """[Intro]
"Look, I was gonna go easy on you not to hurt your feelings"
"But I'm only going to get this one chance" (Six minutes—, six minutes—)
"Something's wrong, I can feel it" (Six minutes, Slim Shady, you're on!)
"Just a feeling I've got, like something's about to happen, but I don't know what. 
If that means what I think it means, we're in trouble, big trouble; 
And if he is as bananas as you say, I'm not taking any chances"
"You are just what the doc ordered"

[Chorus]
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot

[Verse 1]
But for me to rap like a computer it must be in my genes
I got a laptop in my back pocket
My pen'll go off when I half-cock it
Got a fat knot from that rap profit
Made a livin' and a killin' off it
Ever since Bill Clinton was still in office
With Monica Lewinsky feelin' on his nutsack
I'm an MC still as honest
But as rude and as indecent as all hell
Syllables, skill-a-holic (Kill 'em all with)
This flippity dippity-hippity hip-hop
You don't really wanna get into a pissin' match
With this rappity brat, packin' a MAC in the back of the Ac'
Backpack rap crap, yap-yap, yackety-yack
And at the exact same time, I attempt these lyrical acrobat stunts while I'm practicin' that
I'll still be able to break a motherfuckin' table
Over the back of a couple of faggots and crack it in half
Only realized it was ironic, I was signed to Aftermath after the fact
How could I not blow? All I do is drop F-bombs
Feel my wrath of attack
Rappers are havin' a rough time period, here's a maxi pad
It's actually disastrously bad for the wack
While I'm masterfully constructing this masterpièce

[Chorus]
'Cause I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
Let me show you maintainin' this shit ain't that hard, that hard
Everybody want the key and the secret to rap immortality like Ι have got

[Verse 2]
Well, to be truthful the blueprint's
Simply rage and youthful exuberance
Everybody loves to root for a nuisance
Hit the Earth like an asteroid
Did nothing but shoot for the Moon since (Pew!)
MCs get taken to school with this music
'Cause I use it as a vehicle to "bus the rhyme"
Now I lead a new school full of students
Me? I'm a product of Rakim
Lakim Shabazz, 2Pac, N.W.A, Cube, hey Doc, Ren
Yella, Eazy, thank you, they got Slim
Inspired enough to one day grow up, blow up and be in a position
To meet Run–D.M.C., and induct them
Into the motherfuckin' Rock and Roll Hall of Fame
Even though I'll walk in the church and burst in a ball of flames
Only Hall of Fame I'll be inducted in is the alcohol of fame
On the wall of shame
You fags think it's all a game, 'til I walk a flock of flames
Off a plank and, tell me what in the fuck are you thinkin'?
Little gay-lookin' boy
So gay I can barely say it with a straight face, lookin' boy (Ha-ha!)
You're witnessin' a mass-occur
Like you're watching a church gathering take place, lookin' boy
"Oy vey, that boy's gay!"—that's all they say, lookin' boy
You get a thumbs up, pat on the back
And a "way to go" from your label every day, lookin' boy
Hey, lookin' boy! What you say, lookin' boy?
I get a "hell yeah" from Dre, lookin' boy
I'ma work for everything I have, never asked nobody for shit
Get outta my face, lookin' boy!
Basically, boy, you're never gonna be capable
Of keepin' up with the same pace, lookin' boy, 'cause—

[Chorus]
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
The way I'm racin' around the track, call me NASCAR, NASCAR
Dale Earnhardt of the trailer park, the White Trash God
Kneel before General Zod
This planet's Krypton—no, Asgard, Asgard

[Verse 3]
So you'll be Thor and I'll be Odin
You rodent, I'm omnipotent
Let off, then I'm reloadin'
Immediately with these bombs I'm totin'
And I should not be woken
I'm the walkin' dead, but I'm just a talkin' head, a zombie floatin'
But I got your mom deep-throatin'
I'm out my Ramen Noodle
We have nothin' in common, poodle
I'm a Doberman, pinch yourself in the arm and pay homage, pupil
It's me, my honesty's brutal
But it's honestly futile if I don't utilize what I do though
For good at least once in a while
So I wanna make sure somewhere in this chicken scratch I scribble and doodle enough rhymes
To maybe try to help get some people through tough times
But I gotta keep a few punchlines
Just in case 'cause even you unsigned
Rappers are hungry lookin' at me like it's lunchtime
I know there was a time where once I
Was king of the underground
But I still rap like I'm on my Pharoahe Monch grind
So I crunch rhymes, but sometimes when you combine
Appeal with the skin color of mine
You get too big and here they come tryin'
To censor you like that one line
I said on "I'm Back" from The Mathers LP 1 when I
Tried to say I'll take seven kids from Columbine
Put 'em all in a line, add an AK-47, a revolver and a 9
See if I get away with it now that I ain't as big as I was, but I'm
Morphin' into an immortal, comin' through the portal
You're stuck in a time warp from 2004 though
And I don't know what the fuck that you rhyme for
You're pointless as Rapunzel with fuckin' cornrows
You write normal? Fuck being normal!
And I just bought a new raygun from the future
Just to come and shoot ya, like when Fabolous made Ray J mad
'Cause Fab said he looked like a fag at Mayweather's pad
Singin' to a man while he played piano
Man, oh man, that was a 24/7 special on the cable channel
So Ray J went straight to the radio station
The very next day, "Hey Fab, I'ma kill you!"
Lyrics comin' at you at supersonic speed (J.J. Fad)
Uh, summa-lumma, dooma-lumma, you assumin' I'm a human
What I gotta do to get it through to you? I'm superhuman
Innovative and I'm made of rubber so that anything you say is ricochetin' off of me and it'll glue to you and
I'm devastating, more than ever demonstrating
How to give a motherfuckin' audience a feeling like it's levitating
Never fading, and I know the haters are forever waiting
For the day that they can say I fell off, they'll be celebrating
'Cause I know the way to get 'em motivated
I make elevating music, you make elevator music
"Oh, he's too mainstream."
Well, that's what they do when they get jealous, they confuse it
"It's not hip-hop, it's pop,"—'cause I found a hella way to fuse it
With rock, shock rap with Doc
Throw on "Lose Yourself" and make 'em lose it
"I don't know how to make songs like that
I don't know what words to use."
Let me know when it occurs to you
While I'm rippin' any one of these verses that versus you
It's curtains, I'm inadvertently hurtin' you
How many verses I gotta murder to
Prove that if you were half as nice, your songs you could sacrifice virgins too?!
Ugh, school flunky, pill junkie
But look at the accolades these skills brung me
Full of myself, but still hungry
I bully myself 'cause I make me do what I put my mind to
And I'm a million leagues above you
Ill when I speak in tongues, but it's still tongue-in-cheek, fuck you
I'm drunk, so, Satan, take the fucking wheel
I'ma sleep in the front seat
Bumpin' Heavy D and the Boyz, still "Chunky but Funky"
But in my head there's something I can feel tugging and struggling
Angels fight with devils and here's what they want from me
They're askin' me to eliminate some of the women hate
But if you take into consideration the bitter hatred
I have, then you may be a little patient
And more sympathetic to the situation
And understand the discrimination
But fuck it, life's handin' you lemons? Make lemonade then!
But if I can't batter the women
How the fuck am I supposed to bake 'em a cake then?
Don't mistake him for Satan
It's a fatal mistake if you think I need to be overseas and take a vacation
To trip a broad, and make her fall on her face and
Don't be a retard—be a king? Think not
Why be a king when you can be a god?"""

In [ ]:
lyrics = taylor_lyrics
lyrics = re.sub(r"\[.*?\]", "", lyrics, flags=re.MULTILINE)
lyrics = re.sub(r"’", "'", lyrics)
lyrics = re.sub(r"[^a-zA-Z'’|-]|\s", "|", lyrics)
transcript = lyrics.upper()
transcript

In [ ]:
dictionary = {c: i for i, c in enumerate(labels)}

tokens = [dictionary[c] for c in transcript]
print(list(zip(transcript, tokens)))


def get_trellis(emission, tokens, blank_id=0):
    num_frame = emission.size(0)
    num_tokens = len(tokens)

    # Trellis has extra diemsions for both time axis and tokens.
    # The extra dim for tokens represents <SoS> (start-of-sentence)
    # The extra dim for time axis is for simplification of the code.
    trellis = torch.empty((num_frame + 1, num_tokens + 1))
    trellis[0, 0] = 0
    trellis[1:, 0] = torch.cumsum(emission[:, 0], 0)
    trellis[0, -num_tokens:] = -float("inf")
    trellis[-num_tokens:, 0] = float("inf")

    for t in range(num_frame):
        trellis[t + 1, 1:] = torch.maximum(
            # Score for staying at the same token
            trellis[t, 1:] + emission[t, blank_id],
            # Score for changing to the next token
            trellis[t, :-1] + emission[t, tokens],
        )
    return trellis


trellis = get_trellis(emission, tokens)

## Find the most likely path (backtracking)

Once the trellis is generated, we will traverse it following the
elements with high probability.

We will start from the last label index with the time step of highest
probability, then, we traverse back in time, picking stay
($c_j \rightarrow c_j$) or transition
($c_j \rightarrow c_{j+1}$), based on the post-transition
probability $k_{t, j} p(t+1, c_{j+1})$ or
$k_{t, j+1} p(t+1, repeat)$.

Transition is done once the label reaches the beginning.

The trellis matrix is used for path-finding, but for the final
probability of each segment, we take the frame-wise probability from
emission matrix.




In [ ]:
@dataclass
class Point:
    token_index: int
    time_index: int
    score: float


def backtrack(trellis, emission, tokens, blank_id=0):
    # Note:
    # j and t are indices for trellis, which has extra dimensions
    # for time and tokens at the beginning.
    # When referring to time frame index `T` in trellis,
    # the corresponding index in emission is `T-1`.
    # Similarly, when referring to token index `J` in trellis,
    # the corresponding index in transcript is `J-1`.
    j = trellis.size(1) - 1
    t_start = torch.argmax(trellis[:, j]).item()

    path = []
    for t in range(t_start, 0, -1):
        # 1. Figure out if the current position was stay or change
        # Note (again):
        # `emission[J-1]` is the emission at time frame `J` of trellis dimension.
        # Score for token staying the same from time frame J-1 to T.
        stayed = trellis[t - 1, j] + emission[t - 1, blank_id]
        # Score for token changing from C-1 at T-1 to J at T.
        changed = trellis[t - 1, j - 1] + emission[t - 1, tokens[j - 1]]

        # 2. Store the path with frame-wise probability.
        prob = emission[t - 1, tokens[j - 1] if changed > stayed else 0].exp().item()
        # Return token index and time index in non-trellis coordinate.
        path.append(Point(j - 1, t - 1, prob))

        # 3. Update the token
        if changed > stayed:
            j -= 1
            if j == 0:
                break
    else:
        raise ValueError("Failed to align")
    return path[::-1]


path = backtrack(trellis, emission, tokens)
# for p in path:
#     print(p)

### Visualization



In [ ]:
def plot_trellis_with_path(trellis, path):
    # To plot trellis with path, we take advantage of 'nan' value
    trellis_with_path = trellis.clone()
    for _, p in enumerate(path):
        trellis_with_path[p.time_index, p.token_index] = float("nan")
    plt.imshow(trellis_with_path[1:, 1:].T, origin="lower")


plot_trellis_with_path(trellis, path)
plt.title("The path found by backtracking")
plt.show()

Looking good. Now this path contains repetations for the same labels, so
let’s merge them to make it close to the original transcript.

When merging the multiple path points, we simply take the average
probability for the merged segments.




In [ ]:
# Merge the labels
@dataclass
class Segment:
    label: str
    start: int
    end: int
    score: float

    def __repr__(self):
        return f"{self.label}\t({self.score:4.2f}): [{self.start:5d}, {self.end:5d})"

    @property
    def length(self):
        return self.end - self.start


def merge_repeats(path):
    i1, i2 = 0, 0
    segments = []
    while i1 < len(path):
        while i2 < len(path) and path[i1].token_index == path[i2].token_index:
            i2 += 1
        score = sum(path[k].score for k in range(i1, i2)) / (i2 - i1)
        segments.append(
            Segment(
                transcript[path[i1].token_index],
                path[i1].time_index,
                path[i2 - 1].time_index + 1,
                score,
            )
        )
        i1 = i2
    return segments


segments = merge_repeats(path)
# for seg in segments:
#     print(seg)

### Visualization



Looks good. Now let’s merge the words. The Wav2Vec2 model uses ``'|'``
as the word boundary, so we merge the segments before each occurance of
``'|'``.

Then, finally, we segment the original audio into segmented audio and
listen to them to see if the segmentation is correct.




In [ ]:
# Merge words
def merge_words(segments, separator="|"):
    words = []
    i1, i2 = 0, 0
    while i1 < len(segments):
        if i2 >= len(segments) or segments[i2].label == separator:
            if i1 != i2:
                segs = segments[i1:i2]
                word = "".join([seg.label for seg in segs])
                score = sum(seg.score * seg.length for seg in segs) / sum(seg.length for seg in segs)
                words.append(Segment(word, segments[i1].start, segments[i2 - 1].end, score))
            i1 = i2 + 1
            i2 = i1
        else:
            i2 += 1
    return words


word_segments = merge_words(segments)
for word in word_segments:
    print(word)

In [ ]:
def calculate_timestamps(dict, timestamp, timestamp_old):
    ratio = waveform.size(1) / (trellis.size(0) - 1)
    i = 0
    for word in word_segments:
      x0 = int(ratio * word.start)
      x1 = int(ratio * word.end)
      timestamp_full = x0 / bundle.sample_rate
      if i == 0:
        timestamp = timestamp_full + timestamp_old
      else:  
        timestamp = timestamp_full - timestamp_old
      dict.append((timestamp, word.label))
      timestamp_old = timestamp_full
      i += 1
    timestamp_old = (waveform.size(1)/16000) - timestamp_full
    return timestamp, timestamp_old


dict = []
timestamp = 0
timestamp_old = 0
timestamp, timestamp_old = calculate_timestamps(dict, timestamp, timestamp_old)

In [ ]:
from IPython.display import Audio, display
def play_audio(waveform, sample_rate):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  if num_channels == 1:
    display(Audio(waveform[0], rate=sample_rate))
  elif num_channels == 2:
    display(Audio((waveform[0], waveform[1]), rate=sample_rate))
  else:
    raise ValueError("Waveform with more than 2 channels are not supported.")

play_audio(waveform, 16000)

In [ ]:
# import time
# for tuple in dict:
#   time.sleep(tuple[0])
#   print(tuple[1])

### Visualization



In [ ]:
# A trick to embed the resulting audio to the generated file.
# `IPython.display.Audio` has to be the last call in a cell,
# and there should be only one call par cell.
def display_segment(i):
    ratio = waveform.size(1) / (trellis.size(0) - 1)
    word = word_segments[i]
    x0 = int(ratio * word.start)
    x1 = int(ratio * word.end)
    print(f"{word.label} ({word.score:.2f}): {x0 / bundle.sample_rate:.3f} - {x1 / bundle.sample_rate:.3f} sec")
    segment = waveform[:, x0:x1]
    return IPython.display.Audio(segment.numpy(), rate=bundle.sample_rate)

In [ ]:
word_segments

In [ ]:
# Generate the audio for each segment
print(transcript)
# IPython.display.Audio(audio, rate=sr)

In [ ]:
display_segment(0)

In [ ]:
ratio = waveform.size(1) / (trellis.size(0) - 1)

start_at = 0
end_at = start_at + 30

x0 = int(ratio * word_segments[start_at].start)
x1 = int(ratio * word_segments[end_at].end)
segment = waveform[:, x0:x1]
print(list(map(lambda x: x.label, word_segments[start_at:end_at + 1])))
IPython.display.Audio(segment.numpy(), rate=bundle.sample_rate)

## Conclusion

In this tutorial, we looked how to use torchaudio’s Wav2Vec2 model to
perform CTC segmentation for forced alignment.




In [ ]:
!pip install jupyter-dash
!pip install pytube

# Functions

In [ ]:
import re

In [ ]:
def extract_video_id(link):
    regExp = re.compile(r'^.*(youtu.be\/|v\/|u\/\w\/|embed\/|watch\?v=|&v=)([^#&?]*).*')
    match_id = regExp.match(link)
    if match_id:
        video_id = match_id.group(2)
        print(video_id)
        if len(video_id) == 11:
            return video_id 
    return None

# Website

In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [ ]:
app.layout = html.Div([
    html.H1("AI Karaoke"),
    # Yt link
    html.Div([
        dcc.Input(id="input_yt", placeholder="Input Youtube Link", style={'width': '600px', 'margin-right': '5px'}),
        html.Button('Submit', id='btn_submit', n_clicks=0),
        html.Div(id="initial_message", children="Enter a YouTube link and transcript and press submit to load video", style={'display': 'block'}),
        html.Div(id="invalid_link_div", children="Invalid YouTube link", style={'color': 'red', 'display': 'none'}),
    ], style={'margin-bottom': '30px'}),
     
    # video and transcript
    html.Div([
        html.Iframe(width="70%", height="80%", id='iframe_video', title="YouTube video player", allow="accelerometer, autoplay, clipboard-write, encrypted-media, gyroscope, picture-in-picture ", style={'display': 'inline-block'}),
        # TODO change input as you need
        dcc.Input(id="input_transcript", placeholder="Input transcript", style={'width': '29%', 'height': '80%', 'display': 'inline-block'})
    ], style={'height': '100vh', 'width': '100%', 'display': 'flex'})
     
])

@app.callback(
    [Output('iframe_video', 'src'),
    Output('initial_message', 'style'),
    Output('invalid_link_div', 'style')],
    Input('btn_submit', 'n_clicks'),
    State('input_yt', 'value')
)
def embed_video(n_clicks, link):
    initial_message_style = {'display': 'block'}
    invalid_style = {'color': 'red', 'display': 'none'}
    src = ""
    if n_clicks > 0:
        video_id = extract_video_id(link)
        if video_id:
            src = f"https://www.youtube.com/embed/{video_id}"
        else:
            initial_message_style = {'display': 'none'}
            invalid_style = {'color': 'red', 'display': 'block'}
    return src, initial_message_style, invalid_style

# TODO complete this callback for displaying lyrics (with highlight)
# TODO add State for current video timestamp (so if the user jumps inside the video, it shows the lyrics of the selected timestamp)
@app.callback(
    Output('input_transcript', 'value'),
    Input('btn_submit', 'n_clicks'),
    State('input_transcript', 'value'),
    State('input_yt', 'value')
)
def compute_force_alignment(n_clicks, transcript, link):
    value = transcript
    if n_clicks > 0:
      #TODO compute alignment and current display of transcript
      pass
    return value

In [ ]:
# click link to open website in new tab
if __name__ == '__main__':
    app.run_server(mode='inline')